<!-- # HSI deeplearning FYP -->

In [2]:
import torch
from torch import nn, utils
import torchvision
from torch.utils.data import ConcatDataset, DataLoader, random_split
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import os
import scipy.io 
import numpy as np
import cv2
import random
from sklearn.decomposition import PCA
from sklearn.utils import shuffle
from scipy.signal import savgol_filter
import pandas as pd #don't work with newest numpy
from imblearn.over_sampling import SMOTE
from PIL import Image


<!-- # Quality control: Remove low intensity spectra, PCA (80), removing paraffin wax regions -->

In [ ]:
from sklearn.decomposition import PCA
import numpy as np
from scipy.signal import savgol_filter

def pca(spectral_image, n_components=80):
    """
    Apply PCA for noise reduction on a hyperspectral image.

    Parameters:
    spectral_image (numpy array): 3D array (H, W, C) representing spectral data
    n_components (int): Number of principal components to retain

    Returns:
    numpy array: Denoised spectral image (H, W, C)
    """
    H, W, C = spectral_image.shape
    reshaped_spectra = spectral_image.reshape(-1, C)  # Flatten to (H*W, C)

    # Compute mean spectrum
    mean_spectra = np.mean(reshaped_spectra, axis=0)

    # Apply PCA
    pca_model = PCA(n_components=n_components)
    transformed = pca_model.fit_transform(reshaped_spectra - mean_spectra)  # Center data
    denoised = pca_model.inverse_transform(transformed) + mean_spectra  # Reconstruct

    # Reshape back to (H, W, C)
    return denoised.reshape(H, W, C)


def preprocess(spectral_image, wavenumbers):
    """
    Preprocess a hyperspectral image with noise reduction and filtering.

    Parameters:
    spectral_image (numpy array): 3D array (H, W, C)
    wavenumbers (numpy array): 1D array of spectral bands

    Returns:
    numpy array: Preprocessed spectral image (H, W, filtered_C)
    """
    H, W, C = spectral_image.shape

    # 1. Quality filtering based on Amide I Band (1600 - 1700 cm⁻¹)
    amide_band_indices = np.where((wavenumbers >= 1600) & (wavenumbers <= 1700))[0]
    amide_spectra = np.max(spectral_image[:, :, amide_band_indices], axis=2)

    # Retain pixels with valid absorbance (0.1 to 2)
    valid_mask = (amide_spectra >= 0.1) & (amide_spectra <= 2)
    
    # Filter out invalid pixels (set spectra to zero where invalid)
    filtered_spectral_image = spectral_image.copy()
    filtered_spectral_image[~valid_mask] = 0  

    # 2. Remove paraffin wax regions
    valid_wavenumbers = (
        ((wavenumbers >= 1000) & (wavenumbers <= 1319)) | 
        ((wavenumbers >= 1481) & (wavenumbers <= 1769)) |
        ((wavenumbers >= 2986) & (wavenumbers <= 3569))
    )

    # Apply filtering to keep only valid wavenumbers
    filtered_spectral_image = filtered_spectral_image[:, :, valid_wavenumbers]
    filtered_wavenumbers = wavenumbers[valid_wavenumbers]

    # 3. Compute First Derivative Using Savitzky-Golay Filter
    window_size = 19  
    poly_order = 4
    for i in range(H):
        for j in range(W):
            filtered_spectral_image[i, j, :] = savgol_filter(filtered_spectral_image[i, j, :], window_length=window_size, polyorder=poly_order, deriv=1)

    # 4. Remove Edge Regions Influenced by Derivatization
    valid_end_regions = (
        ((filtered_wavenumbers >= 1019) & (filtered_wavenumbers <= 1300)) |
        ((filtered_wavenumbers >= 1500) & (filtered_wavenumbers <= 1750)) |
        ((filtered_wavenumbers >= 3005) & (filtered_wavenumbers <= 3550))
    )

    final_spectral_image = filtered_spectral_image[:, :, valid_end_regions]
    
    return final_spectral_image


## Testing diff shape for 3D CNN deepseek

In [ ]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import gc

# Define file locations
data_location = r"C:\Users\Edmund Chia\Desktop\HSI-cancer-fyp\DataCube\DataCube_npz"
annotation_location = r"C:\Users\Edmund Chia\Desktop\HSI-cancer-fyp\overlay"

core_names = [
    "A12","C10", "C16", "C5", "C8",
    "D5", "D7", "D8", "D9", "E7", "E8", "F5", "F7", "F8",
    "G6", "H7", "H10", "H9", "I3", "I5", "A14", "I7", "B9", "I10",
    "K1", "K10", "K6", "K9", "B11", "L10", "L2", "L3",
    "M1", "M3", "M4", "M6", "M9", "M10", "M11", "M13"
]

annotation_colours = {
    "red": (255, 0, 0),      # Cancer
    "green": (103, 193, 66), # Normal
    "orange": (243, 143, 53),# Normal
    "purple": (165, 55, 156) # Cancer
}

# Function to extract patches
def extract_patches(hyperspectral_cube, label_map, patch_size=32):
    patches = []
    labels = []
    height, width, spectral_bands = hyperspectral_cube.shape

    for i in range(0, height - patch_size + 1, patch_size // 2):  # Sliding window with 50% overlap
        for j in range(0, width - patch_size + 1, patch_size // 2):
            patch = hyperspectral_cube[i:i+patch_size, j:j+patch_size, :]
            label_patch = label_map[i:i+patch_size, j:j+patch_size]

            # Only include patches with at least one annotated pixel
            if np.any(label_patch != -1):  # -1 indicates unannotated pixels
                patches.append(patch)
                labels.append(label_patch)

    return np.array(patches, dtype=np.float32), np.array(labels, dtype=np.float32)  # Reduce precision

# Lists to store data
X = []  # Input data (patches of hyperspectral cubes)
Y = []  # Labels (patches of label maps)

# Process each core in smaller batches
batch_size = 5  # Process 5 cores at a time
for i in range(0, len(core_names), batch_size):
    batch_cores = core_names[i:i + batch_size]
    batch_X = []
    batch_Y = []

    for core_name in batch_cores:
        mat_path = os.path.join(data_location, core_name + ".npz")
        img_path = os.path.join(annotation_location, core_name + ".png")

        # Load hyperspectral data
        data = np.load(mat_path)
        spectra = data["spectra"].astype(np.float32)  # Reduce precision
        wavenumbers = data["wavenumbers"]
        indices = data["indices"].flatten()

        # Load annotated image
        annotated_img = Image.open(img_path)
        annotated_img = np.array(annotated_img)

        # Get spatial dimensions
        ypixels, xpixels = annotated_img.shape[:2]

        # Convert indices to (row, col) positions
        rows = indices % ypixels
        cols = indices // ypixels

        # Create a 3D hyperspectral cube
        hyperspectral_cube = np.zeros((ypixels, xpixels, spectra.shape[1]), dtype=np.float32)  # Reduce precision
        hyperspectral_cube[rows, cols, :] = spectra  # Fill the cube with spectra



    # Append batch data to main lists
    X.append(np.concatenate(batch_X, axis=0))
    Y.append(np.concatenate(batch_Y, axis=0))

    # Clear batch variables
    del batch_X, batch_Y
    gc.collect()

# Convert lists to NumPy arrays
X = np.concatenate(X, axis=0)  # Shape: [num_patches, patch_size, patch_size, spectral_bands]
Y = np.concatenate(Y, axis=0)  # Shape: [num_patches, patch_size, patch_size]

# Print shapes
print("Hyperspectral patches shape:", X.shape)
print("Label patches shape:", Y.shape)




In [ ]:
## for gpt 3D pca
import numpy as np
from sklearn.decomposition import PCA
from scipy.signal import savgol_filter

def pca(spectral_data, n_components=80):
    """
    Apply PCA for noise reduction on a hyperspectral image or spectral data.

    Parameters:
    spectral_data (numpy array): 2D array (N, C) or 3D array (H, W, C) of spectral data
    n_components (int): Number of principal components to retain

    Returns:
    numpy array: Denoised spectral data with the same shape as input
    """
    # Check input shape
    if spectral_data.ndim == 3:  # If input is (H, W, C)
        H, W, C = spectral_data.shape
        reshaped_spectra = spectral_data.reshape(-1, C)  # Flatten to (H*W, C)
    elif spectral_data.ndim == 2:  # If input is (N, C)
        reshaped_spectra = spectral_data
    else:
        raise ValueError(f"Invalid input shape {spectral_data.shape}, expected 2D (N, C) or 3D (H, W, C)")

    # Compute mean spectrum
    mean_spectra = np.mean(reshaped_spectra, axis=0)

    # Apply PCA
    pca_model = PCA(n_components=n_components)
    try:
        transformed = pca_model.fit_transform(reshaped_spectra - mean_spectra)  # Center data
        denoised = pca_model.inverse_transform(transformed) + mean_spectra  # Reconstruct
    except Exception as e:
        print(f"Error during PCA transformation: {e}")
        return None

    # Ensure the result is a valid NumPy array
    if not isinstance(denoised, np.ndarray):
        print(f"PCA output is not a valid numpy array. Type: {type(denoised)}")
        return None

    # Reshape back if input was 3D
    if spectral_data.ndim == 3:
        return denoised.reshape(H, W, -1)
    
    return denoised


def preprocess(spectral_images, wavenumbers):
    """
    Preprocess hyperspectral images with memory-efficient noise reduction and filtering.

    Parameters:
    spectral_images (numpy array): 4D array (N, H, W, C)
    wavenumbers (numpy array): 1D array of spectral bands

    Returns:
    numpy array: Preprocessed spectral images with reduced spectral channels
    """
    N, H, W, C = spectral_images.shape
    final_images = []

    for i in range(N):  # Process one image at a time
        image = spectral_images[i].astype(np.float16)  # Convert to float16 to save memory

        # 1. Quality filtering based on Amide I Band (1600 - 1700 cm⁻¹)
        amide_band_indices = np.where((wavenumbers >= 1600) & (wavenumbers <= 1700))[0]
        amide_spectra = np.max(image[:, :, amide_band_indices], axis=-1)

        # Retain pixels with valid absorbance (0.1 to 2)
        valid_mask = (amide_spectra >= 0.1) & (amide_spectra <= 2)
        image[~valid_mask, :] = 0  # Zero out invalid pixels

        # 2. Remove paraffin wax regions
        valid_wavenumbers = np.where(
            ((wavenumbers >= 1000) & (wavenumbers <= 1319)) | 
            ((wavenumbers >= 1481) & (wavenumbers <= 1769)) |
            ((wavenumbers >= 2986) & (wavenumbers <= 3569))
        )[0]

        # Apply filtering
        image = image[:, :, valid_wavenumbers]
        filtered_wavenumbers = wavenumbers[valid_wavenumbers]

        # 3. Compute First Derivative Using Savitzky-Golay Filter (Apply Along Last Axis)
        window_size = 19  
        poly_order = 4
        for x in range(H):
            for y in range(W):
                image[x, y, :] = savgol_filter(
                    image[x, y, :], window_length=window_size, polyorder=poly_order, deriv=1, 
                    mode="nearest"
                )

        # 4. Remove Edge Regions Influenced by Derivatization
        valid_end_regions = np.where(
            ((filtered_wavenumbers >= 1019) & (filtered_wavenumbers <= 1300)) |
            ((filtered_wavenumbers >= 1500) & (filtered_wavenumbers <= 1750)) |
            ((filtered_wavenumbers >= 3005) & (filtered_wavenumbers <= 3550))
        )[0]

        final_image = image[:, :, valid_end_regions]
        final_images.append(final_image)  # Store processed image

    # Convert list back to numpy array
    return np.array(final_images), filtered_wavenumbers[valid_end_regions]





## GPT label and spectra

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.decomposition import PCA

# Define constants
TARGET_H, TARGET_W = 256, 256  # Standardized spatial size
DTYPE = np.float32  # Use lower precision

def pad_or_crop(image, target_size=(256, 256), fill_value=0):
    """
    Resize an image to a fixed size by padding or cropping.
    """
    h, w = image.shape[:2]
    target_h, target_w = target_size

    # Initialize a blank canvas
    new_image = np.full((target_h, target_w, image.shape[-1]), fill_value, dtype=image.dtype)

    # Determine cropping or centering
    crop_h, crop_w = min(h, target_h), min(w, target_w)
    start_h, start_w = (h - crop_h) // 2, (w - crop_w) // 2
    target_start_h, target_start_w = (target_h - crop_h) // 2, (target_w - crop_w) // 2

    # Crop and place in the center
    new_image[target_start_h:target_start_h + crop_h, target_start_w:target_start_w + crop_w] = \
        image[start_h:start_h + crop_h, start_w:start_w + crop_w]

    return new_image


# Define file locations
data_location = r"C:\Users\Edmund Chia\Desktop\HSI-cancer-fyp\DataCube\DataCube_npz"
annotation_location = r"C:\Users\Edmund Chia\Desktop\HSI-cancer-fyp\overlay"

core_names = [
    "A12","C10", "C16"
    # , "C5", "C8",
    # "D5", "D7", "D8", "D9", "E7", "E8", "F5", "F7", "F8",
    # "G6", "H7", "H10", "H9", "I3", "I5", "A14", "I7", "B9", "I10",
    # "K1", "K10", "K6", "K9", "B11", "L10", "L2", "L3",
    # "M1", "M3", "M4", "M6", "M9", "M10", "M11", "M13"
]

annotation_colours = {
    "red": (255, 0, 0),      # Cancer
    "green": (103, 193, 66), # Normal
    "orange": (243, 143, 53),# Normal
    "purple": (165, 55, 156) # Cancer
}


# Process each core ensuring uniform shape
all_spectral_images = []
all_label_masks = []

for core_name in core_names:
    mat_path = os.path.join(data_location, core_name + ".npz")
    img_path = os.path.join(annotation_location, core_name + ".png")

    data = np.load(mat_path)
    spectra = data["spectra"].astype(DTYPE)  # Convert early to lower precision
    wavenumbers = data["wavenumbers"]
    indices = data["indices"].flatten()

    annotated_img = np.array(Image.open(img_path))
    ypixels, xpixels = annotated_img.shape[:2]
    spectral_channels = spectra.shape[1]

    # Convert indices to (row, col) positions
    rows = indices % ypixels  
    cols = indices // ypixels

    # Create empty spectral image grid and label mask
    spectral_image = np.zeros((ypixels, xpixels, spectral_channels), dtype=DTYPE)



    # Convert to NumPy arrays
    selected_spectra = np.array(selected_spectra, dtype=DTYPE)
    selected_labels = np.array(selected_labels)

    # Apply PCA for noise reduction
    denoised_spectra = pca(selected_spectra)

    # Create spatial mask
    for i, (row, col) in enumerate(selected_positions):
        spectral_image[row, col, :] = denoised_spectra[i]
        label_mask[row, col] = selected_labels[i]

    # Ensure uniform shape (256x256)
    spectral_image = pad_or_crop(spectral_image, (TARGET_H, TARGET_W), fill_value=0)
    label_mask = pad_or_crop(label_mask[..., np.newaxis], (TARGET_H, TARGET_W), fill_value=0)[..., 0]  # Remove extra channel

    # Append results (or save to disk to free memory)
    all_spectral_images.append(spectral_image)
    all_label_masks.append(label_mask)

# Convert to NumPy arrays
all_spectral_images = np.array(all_spectral_images, dtype=DTYPE)
all_label_masks = np.array(all_label_masks, dtype=np.uint8)

# Print final shapes
print(f"Spectral Image Shape (for CNN input): {all_spectral_images.shape}")  # (N, 256, 256, C)
print(f"Label Mask Shape: {all_label_masks.shape}")  # (N, 256, 256)



In [ ]:
import gc

# Free memory before processing
del selected_spectra, selected_labels, selected_positions, spectral_image, label_mask
gc.collect()  # Force garbage collection


In [ ]:
import gc

batch_size = 5  # Process only 5 images at a time (adjust if needed)
X_list = []

for i in range(0, len(all_spectral_images), batch_size):
    batch = all_spectral_images[i:i + batch_size]  # Get batch
    X_batch = preprocess(batch, wavenumbers)  # Process batch
    X_list.append(X_batch)  # Store results
    
    # Free memory
    del batch, X_batch
    gc.collect()

# Convert back to NumPy array
X = np.concatenate(X_list, axis=0)
del X_list  # Free list
gc.collect()

print(X.shape)


In [16]:
# for correct method
# import numpy as np
from sklearn.decomposition import PCA
from scipy.signal import savgol_filter

from sklearn.decomposition import PCA
import numpy as np

def pca_denoise(spectral_data, variance_threshold=0.99):
    if spectral_data.ndim != 3:
        raise ValueError(f"Expected (H, W, C) shape, not {spectral_data.shape}")

    H, W, C = spectral_data.shape
    reshaped_spectra = spectral_data.reshape(-1, C)  # Flatten to (H*W, C)

    # Handle NaNs/Infs before PCA
    reshaped_spectra = np.nan_to_num(reshaped_spectra)

    # Apply PCA
    pca = PCA(n_components=variance_threshold, whiten=True)
    transformed = pca.fit_transform(reshaped_spectra)  # Auto-centers data
    denoised = pca.inverse_transform(transformed)  # Reconstruct
    
    return denoised.reshape(H, W, C)  # Restore original shape



def preprocess(spectral_images, wavenumbers):
    """
    Preprocess hyperspectral images with memory-efficient noise reduction and filtering.

    Parameters:
    spectral_images (numpy array): 4D array (N, H, W, C)
    wavenumbers (numpy array): 1D array of spectral bands

    Returns:
    numpy array: Preprocessed spectral images with reduced spectral channels
    """
    N, H, W, C = spectral_images.shape
    final_images = []

    for i in range(N):  # Process one image at a time
        image = spectral_images[i].astype(np.float16)  # Convert to float16 to save memory

        # 1. Quality filtering based on Amide I Band (1600 - 1700 cm⁻¹)
        amide_band_indices = np.where((wavenumbers >= 1600) & (wavenumbers <= 1700))[0]
        amide_spectra = np.max(image[:, :, amide_band_indices], axis=-1)

        # Retain pixels with valid absorbance (0.1 to 2)
        valid_mask = (amide_spectra >= 0.1) & (amide_spectra <= 2)
        image[~valid_mask, :] = 0  # Zero out invalid pixels

        # 2. Remove paraffin wax regions
        valid_wavenumbers = np.where(
            ((wavenumbers >= 1000) & (wavenumbers <= 1319)) | 
            ((wavenumbers >= 1481) & (wavenumbers <= 1769)) |
            ((wavenumbers >= 2986) & (wavenumbers <= 3569))
        )[0]

        # Apply filtering
        image = image[:, :, valid_wavenumbers]
        filtered_wavenumbers = wavenumbers[valid_wavenumbers]

        # 3. Compute First Derivative Using Savitzky-Golay Filter (Apply Along Last Axis)
        window_size = 19  
        poly_order = 4
        for x in range(H):
            for y in range(W):
                image[x, y, :] = savgol_filter(
                    image[x, y, :], window_length=window_size, polyorder=poly_order, deriv=1, 
                    mode="nearest"
                )

        # 4. Remove Edge Regions Influenced by Derivatization
        valid_end_regions = np.where(
            ((filtered_wavenumbers >= 1019) & (filtered_wavenumbers <= 1300)) |
            ((filtered_wavenumbers >= 1500) & (filtered_wavenumbers <= 1750)) |
            ((filtered_wavenumbers >= 3005) & (filtered_wavenumbers <= 3550))
        )[0]

        final_image = image[:, :, valid_end_regions]
        final_images.append(final_image)  # Store processed image

    # Convert list back to numpy array
    return np.array(final_images), filtered_wavenumbers[valid_end_regions]





In [ ]:
### CORRECT CONCEPT 



def center_zero_pad(image, target_size=(256, 256)):
    h, w, c = image.shape
    target_h, target_w = target_size

    # Find non-zero (or non-background) pixels
    non_zero_rows = np.where(np.any(image != 0, axis=(1, 2)))[0]
    non_zero_cols = np.where(np.any(image != 0, axis=(0, 2)))[0]

    # Get bounding box of the core region
    min_r, max_r = non_zero_rows[0], non_zero_rows[-1]
    min_c, max_c = non_zero_cols[0], non_zero_cols[-1]

    # Crop to bounding box
    cropped_image = image[min_r:max_r+1, min_c:max_c+1, :]

    # Get new dimensions after cropping
    h_cropped, w_cropped, _ = cropped_image.shape

    # Create a blank canvas filled with zeros
    padded_image = np.zeros((target_h, target_w, c), dtype=np.float32)

    # Compute new centering offsets
    start_h = (target_h - h_cropped) // 2
    start_w = (target_w - w_cropped) // 2

    # Place the cropped image in the center
    padded_image[start_h:start_h + h_cropped, start_w:start_w + w_cropped, :] = cropped_image

    return padded_image


data_location = r"C:\Users\Edmund Chia\Desktop\HSI-cancer-fyp\DataCube\DataCube_npz"
annotation_location = r"C:\Users\Edmund Chia\Desktop\HSI-cancer-fyp\overlay"

core_names = [
    "A12","C10", 
    "C16"
    , "C5", "C8",
    "D5", "D7", "D8", "D9", "E7", "E8", "F5", "F7", "F8",
    "G6", "H7", "H10", "H9", "I3", "I5", "A14", "I7", "B9", "I10",
    "K1", "K10", "K6", "K9", "B11", "L10", "L2", "L3",
    "M1", "M3", "M4", 
    "M6", "M9", "M10", "M11", "M13"
]

annotation_colours = {
    "red": (255, 0, 0),      # Cancer
    "green": (103, 193, 66), # Normal
    "orange": (243, 143, 53),# Normal
    "purple": (165, 55, 156) # Cancer
}

# Lists to store data
X = []  # Input data 

for core_name in core_names:
    mat_path = os.path.join(data_location, core_name + ".npz")
    img_path = os.path.join(annotation_location, core_name + ".png")

    # Load hyperspectral data
    data = np.load(mat_path)
    spectra = data["spectra"] #.astype(np.float32)  # Reduce precision
    wavenumbers = data["wavenumbers"]
    indices = data["indices"].flatten()

    # Load annotated image
    annotated_img = Image.open(img_path)
    annotated_img = np.array(annotated_img)

    # Get spatial dimensions
    ypixels, xpixels = annotated_img.shape[:2]

    # Convert indices to (row, col) positions
    rows = indices % ypixels
    cols = indices // ypixels

    # Create a 3D hyperspectral cube
    spectral_image = np.zeros((ypixels, xpixels, spectra.shape[1]), dtype=np.float32)  # Reduce precision
    spectral_image[rows, cols, :] = spectra  # Fill the cube with spectra
    # print(spectral_image.shape) # should be (H, W, C)
    spectral_image = center_zero_pad(spectral_image)
    # print(spectral_image.shape) # should be (H, W, C)
   
    # plot figure of spectral image for amide 1 band   
    # plt.figure()
    # plt.imshow(spectral_image[:,:,365]) 
    # plt.show()
    clean_spectral_image = pca_denoise(spectral_image)
    # print(clean_spectral_image.shape) # should be (H, W, C)

# # Append batch data to main lists
# X.append(np.concatenate(batch_X, axis=0))
# Y.append(np.concatenate(batch_Y, axis=0))

# # Clear batch variables
# del batch_X, batch_Y
# gc.collect()

# # Convert lists to NumPy arrays
# X = np.concatenate(X, axis=0)  # Shape: [num_patches, patch_size, patch_size, spectral_bands]
# Y = np.concatenate(Y, axis=0)  # Shape: [num_patches, patch_size, patch_size]

# Print shapes
# print("Hyperspectral patches shape:", X.shape)
# print("Label patches shape:", Y.shape)




In [ ]:
 ## !Curr method
# def overlay_extracted_pixels(img, extracted_rows, extracted_cols):
#     overlay_img = img.copy()
#     overlay_img = overlay_img[:,:,:3]
#     for r, c in zip(extracted_rows, extracted_cols):
#         overlay_img[r, c] = (255, 192, 203)  # pink overlay for detected pixels

#     # Display overlay image
#     plt.figure(figsize=(8, 8))
#     plt.imshow(overlay_img)
#     plt.title(f"Overlay of Detected Spectra for {core_name}")
#     plt.axis("off")
#     plt.show()

# # Find annotated pixels
# def find_colour_positions(colour):
#     return set(zip(*np.where(
#         (annotated_img[:, :, 0] == colour[0]) & 
#         (annotated_img[:, :, 1] == colour[1]) &
#         (annotated_img[:, :, 2] == colour[2])
#     )))
    
# # Define file locations
# data_location = r"C:\Users\Edmund Chia\Desktop\HSI-cancer-fyp\DataCube\DataCube_npz"
# annotation_location = r"C:\Users\Edmund Chia\Desktop\HSI-cancer-fyp\overlay"

# core_names = [
#     "A12","C10", "C16", "C5", "C8",
#     "D5", "D7", "D8", "D9", "E7", "E8", "F5", "F7", "F8",
#     "G6", "H7", "H10", "H9", "I3", "I5", "A14", "I7", "B9", "I10",
#     "K1", "K10", "K6", "K9", "B11", "L10", "L2", "L3",
#     "M1", "M3", "M4", "M6", "M9", "M10", "M11", "M13"
# ]

# annotation_colours = {
#     "red": (255, 0, 0),
#     "green": (103, 193, 66),
#     "orange":(243,143,53),
#     "purple":(165,55,156)
# }

# all_spectra = []
# count_spectra = []
# Y = []
    
# # Process each core
# for core_name in core_names:
#     mat_path = os.path.join(data_location, core_name + ".npz")
#     img_path = os.path.join(annotation_location, core_name + ".png")

#     data = np.load(mat_path)

#     spectra = data["spectra"]
#     wavenumbers = data["wavenumbers"]
#     indices = data["indices"].flatten()

#     annotated_img = Image.open(img_path)
#     annotated_img = np.array(annotated_img)
#     # annotated_img = cv2.cvtcolourr(cv2.imread(img_path), cv2.colourr_BGR2RGB) - faster because don't need to convert each from bgr2rgb
#     # print(annotated_img.shape)
    
#     # Fix dimensions (Swapped to match MATLAB)
#     ypixels, xpixels = annotated_img.shape[:2]
#     # ypixels, xpixels = annotated_img.shape[0], annotated_img.shape[1]

#     # Convert indices to (row, col) positions
#     rows = indices % ypixels  
#     cols = indices // ypixels

#     # Convert to NumPy array for consistency
#     pixel_positions = np.column_stack((rows, cols))  
#     # print(len(pixel_positions))

#     red_positions = find_colour_positions(annotation_colours["red"])
#     purple_positions = find_colour_positions(annotation_colours["purple"])
#     green_positions = find_colour_positions(annotation_colours["green"])
#     orange_positions = find_colour_positions(annotation_colours["orange"])
    
#     core_spectra = []
#     extracted_rows = []
#     extracted_cols = []
            
#     for i in range(len(pixel_positions)):
#         row, col = pixel_positions[i]
#         if i >= spectra.shape[0]:  # Prevent index out of bounds
#             continue  

#         if (row, col) in red_positions or (row, col) in purple_positions:
#             core_spectra.append(spectra[i])
#             extracted_rows.append(row)
#             extracted_cols.append(col)
#         elif (row, col) in green_positions or (row, col) in orange_positions:
#             core_spectra.append(spectra[i])
#             extracted_rows.append(row)
#             extracted_cols.append(col)
            

#     # Convert to numpy array
#     core_spectra = np.array(core_spectra)
#     # pca
#     clean_spectra = pca(core_spectra)
#     all_spectra.append(clean_spectra)
    
#     count_spectra.append((core_name, clean_spectra.shape[0]))
 
#     # Overlay detected pixels
#     # overlay_extracted_pixels(annotated_img, extracted_rows, extracted_cols)
# # Convert list to NumPy arrays
# all_spectra = [np.array(core, dtype=np.float32) for core in all_spectra]

# # print(all_spectra)
# # Sort and plot core sizes
# count_spectra.sort(key=lambda x: x[1])
# core_names_sorted, pixel_counts_sorted = zip(*count_spectra)

# plt.figure(figsize=(12, 6))
# plt.bar(core_names_sorted, pixel_counts_sorted, colour="skyblue")

# min_annotated_pixels = pixel_counts_sorted[0]
# # Highlight the core with the minimum pixel count
# min_core_index = pixel_counts_sorted.index(min_annotated_pixels)
# plt.bar(core_names_sorted[min_core_index], pixel_counts_sorted[min_core_index], colour="red", label="Min Core")

# # Add labels and title
# plt.xlabel("Core ID")
# plt.ylabel("Number of Annotated Pixels")
# plt.title("Number of Annotated Pixels per Core")
# plt.xticks(rotation=90)
# plt.legend()
# plt.show()

# print(f"Minimum annotated pixels per core: {pixel_counts_sorted[0]}")

## Getting X and Y

In [ ]:
# # Converting .mat files to .npy files
# training_path = r"C:\Users\Edmund Chia\Desktop\HSI-cancer-fyp\processed-data" 

# os.makedirs(training_path, exist_ok=True)

# # Min_all_spectra is taking the minimum annotated pixels from all cores at random
# min_all_spectra = []

# for core in all_spectra:
#     min_all_spectra.append(core[np.random.choice(core.shape[0],min_annotated_pixels, replace=False)])

# min_all_spectra = np.array(min_all_spectra,dtype=np.float32) # potential change: reshape min_all_spectra to 2d from 3d ( N ,1478)

# wavenumbers = wavenumbers.flatten()
# X = preprocess(min_all_spectra, wavenumbers)
# print("X shape",X.shape) # want this to be (40, something, 558)

# Y = []
# csv_directory = r"C:\Users\Edmund Chia\Desktop\HSI-cancer-fyp\BR20832.csv"
# df = pd.read_csv(csv_directory, usecols=["position", "type"])

# df["type"] = df["type"].str.lower().map({"malignant": 1, "normal": 0, "nat": 0})
# label_dict = df.set_index("position")["type"].to_dict()

# for core in core_names:
#    Y.append(label_dict.get(core))
# # print(Y)
# print("Y shape",len(Y))


# # Save as .npz files
# training_data_path = os.path.join(training_path, 'training.npz')
# np.savez_compressed(training_data_path, X=X, Y=Y)

# print("Saved to training.npz")


## Same for test set

In [ ]:

# core_names = [
#     'L1','I8','H4','E6','C11','F6','F9','I6','M7','M8'
# ]

# annotation_colourrs = {
#     "red": (255, 0, 0),
#     "green": (103, 193, 66),
#     "orange":(243,143,53),
#     "purple":(165,55,156)
# }

# all_spectra = []
# count_spectra = []
# Y = []
    
# # Process each core
# for core_name in core_names:
#     mat_path = os.path.join(data_location, core_name + ".npz")
#     img_path = os.path.join(annotation_location, core_name + ".png")

#     data = np.load(mat_path)

#     spectra = data["spectra"]
#     wavenumbers = data["wavenumbers"]
#     indices = data["indices"].flatten()

#     annotated_img = Image.open(img_path)
#     annotated_img = np.array(annotated_img)
#     # annotated_img = cv2.cvtcolour(cv2.imread(img_path), cv2.colour_BGR2RGB) - faster because don't need to convert each from bgr2rgb
#     # print(annotated_img.shape)
    
#     # Fix dimensions (Swapped to match MATLAB)
#     ypixels, xpixels = annotated_img.shape[:2]
#     # ypixels, xpixels = annotated_img.shape[0], annotated_img.shape[1]

#     # Convert indices to (row, col) positions
#     rows = indices % ypixels  
#     cols = indices // ypixels

#     # Convert to NumPy array for consistency
#     pixel_positions = np.column_stack((rows, cols))  
#     # print(len(pixel_positions))

#     core_spectra = []
#     extracted_rows = []
#     extracted_cols = []
            
#     for i in range(len(pixel_positions)):
#         row, col = pixel_positions[i]
#         if i >= spectra.shape[0]:  # Prevent index out of bounds
#             continue  

#         if (row, col) in red_positions or (row, col) in purple_positions:
#             core_spectra.append(spectra[i])
#             extracted_rows.append(row)
#             extracted_cols.append(col)
#         elif (row, col) in green_positions or (row, col) in orange_positions:
#             core_spectra.append(spectra[i])
#             extracted_rows.append(row)
#             extracted_cols.append(col)
            

#     # Convert to numpy array
#     core_spectra = np.array(core_spectra)
#     # pca
#     clean_spectra = pca(core_spectra)
#     all_spectra.append(clean_spectra)
    
#     count_spectra.append((core_name, clean_spectra.shape[0]))
 
#     # Overlay detected pixels
#     # overlay_extracted_pixels(annotated_img, extracted_rows, extracted_cols)
# # Convert list to NumPy arrays
# all_spectra = [np.array(core, dtype=np.float32) for core in all_spectra]

# # print(all_spectra)
# # Sort and plot core sizes
# count_spectra.sort(key=lambda x: x[1])
# core_names_sorted, pixel_counts_sorted = zip(*count_spectra)

# plt.figure(figsize=(12, 6))
# plt.bar(core_names_sorted, pixel_counts_sorted, colour="skyblue")

# min_annotated_pixels = pixel_counts_sorted[0]
# # Highlight the core with the minimum pixel count
# min_core_index = pixel_counts_sorted.index(min_annotated_pixels)
# plt.bar(core_names_sorted[min_core_index], pixel_counts_sorted[min_core_index], colour="red", label="Min Core")

# # Add labels and title
# plt.xlabel("Core ID")
# plt.ylabel("Number of Annotated Pixels")
# plt.title("Number of Annotated Pixels per Core")
# plt.xticks(rotation=90)
# plt.legend()
# plt.show()

# print(f"Minimum annotated pixels per core: {pixel_counts_sorted[0]}")

# # Converting .mat files to .npy files
# testing_path = r"C:\Users\Edmund Chia\Desktop\HSI-cancer-fyp\processed-data" 

# os.makedirs(testing_path, exist_ok=True)

# # Min_all_spectra is taking the minimum annotated pixels from all cores at random
# min_all_spectra = []

# for core in all_spectra:
#     min_all_spectra.append(core[np.random.choice(core.shape[0],min_annotated_pixels, replace=False)])

# min_all_spectra = np.array(min_all_spectra,dtype=np.float32) # potential change: reshape min_all_spectra to 2d from 3d ( N ,1478)

# wavenumbers = wavenumbers.flatten()
# X = preprocess(min_all_spectra, wavenumbers)
# print("X shape",X.shape) # want this to be (40, something, 558)

# Y = []
# df = pd.read_csv(csv_directory, usecols=["position", "type"])

# df["type"] = df["type"].str.lower().map({"malignant": 1, "normal": 0, "nat": 0})
# label_dict = df.set_index("position")["type"].to_dict()

# for core in core_names:
#    Y.append(label_dict.get(core))
# # print(Y)
# print("Y shape",len(Y))


# # Save as .npz files
# testing_data_path = os.path.join(testing_path, 'testing.npz')
# np.savez_compressed(testing_data_path, X=X, Y=Y)

# print("Saved to testing.npz")


In [ ]:
import torch
print("GPU Available:", torch.cuda.is_available())
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU detected")


In [ ]:
# training_path = r"C:\Users\Edmund Chia\Desktop\HSI-cancer-fyp\processed-data\training.npz" 
# testing_path  = r"C:\Users\Edmund Chia\Desktop\HSI-cancer-fyp\processed-data\testing.npz" 

In [ ]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import DataLoader, Dataset

# data = np.load(training_path, allow_pickle=True)
# X = data["X"]
# Y = data["Y"]

# # print(X)
# # print(Y)

# # Define a 1D CNN for Spectral Data
# class SpectralCNN(nn.Module):
#     def __init__(self, input_dim=558, num_classes=2):
#         super(SpectralCNN, self).__init__()
#         self.conv1 = nn.Conv1d(in_channels=1, out_channels=32, kernel_size=7, stride=1, padding=3)
#         self.bn1 = nn.BatchNorm1d(32)
#         self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=5, stride=1, padding=2)
#         self.bn2 = nn.BatchNorm1d(64)
#         self.conv3 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
#         self.bn3 = nn.BatchNorm1d(128)
#         self.global_avg_pool = nn.AdaptiveAvgPool1d(1)  # Pooling over spectral dimension
#         self.fc = nn.Linear(128, num_classes)

#     def forward(self, x):
#         x = torch.relu(self.bn1(self.conv1(x)))
#         x = torch.relu(self.bn2(self.conv2(x)))
#         x = torch.relu(self.bn3(self.conv3(x)))
#         x = self.global_avg_pool(x).squeeze(-1)  # Global Average Pooling
#         x = self.fc(x)
#         return x

# class SpectraDataset(Dataset):
#     def __init__(self, X, Y):
#         self.data = []
#         self.labels = []
        
#         for i, spectra in enumerate(X):
#             num_pixels = spectra.shape[0]
#             self.data.append(torch.tensor(spectra, dtype=torch.float32))  # Store spectra
#             self.labels.extend([Y[i]] * num_pixels)  # Repeat label for each pixel

#         self.data = torch.cat(self.data, dim=0)  # Stack all spectra
#         self.labels = torch.tensor(self.labels, dtype=torch.long)

#     def __len__(self):
#         return len(self.labels)

#     def __getitem__(self, idx):
#         return self.data[idx].unsqueeze(0), self.labels[idx]  # Add channel dimension


# # Convert X, Y to Dataset
# dataset = SpectraDataset(X, Y)
# train_loader = DataLoader(dataset, batch_size=128, shuffle=True)

# # Initialize model, loss function, optimizer
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = SpectralCNN().to(device)
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)

# # Training loop
# num_epochs = 20
# for epoch in range(num_epochs):
#     model.train()
#     total_loss = 0

#     for batch_X, batch_Y in train_loader:
#         batch_X, batch_Y = batch_X.to(device), batch_Y.to(device)

#         optimizer.zero_grad()
#         outputs = model(batch_X)
#         loss = criterion(outputs, batch_Y)
#         loss.backward()
#         optimizer.step()

#         total_loss += loss.item()

#     print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader):.4f}")


